## 참조자료

https://www.tensorflow.org/guide/function?hl=ko

In [1]:
import tensorflow as tf

## tf.function 데코레이터

tf.function을 함수에 붙여줄 경우, 여전히 다른 일반 함수들처럼 사용할 수 있습니다.

하지만 그래프 내에서 컴파일 되었을 때는 더 빠르게 실행하고, GPU나 TPU를 사용해서 작동하고, 세이브드모델(SavedModel)로 내보내는 것이 가능해집니다.

In [2]:
def simple_nn_layer(x, y):
    return tf.nn.relu(tf.matmul(x, y))


x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))


print("On CPU:")
with tf.device("CPU:0"):
    print(simple_nn_layer(x, y))

On CPU:
tf.Tensor(
[[0.8480972  0.33686098 0.53549767]
 [0.5732886  0.89571023 0.91861194]
 [0.7747946  1.2151299  1.246184  ]], shape=(3, 3), dtype=float32)


In [3]:
simple_nn_layer

<function __main__.simple_nn_layer(x, y)>

In [4]:
@tf.function
def simple_nn_layer(x, y):
    return tf.nn.relu(tf.matmul(x, y))


x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))


print("On CPU:")
with tf.device("CPU:0"):
    print(simple_nn_layer(x, y))

On CPU:
tf.Tensor(
[[0.13940482 0.56941855 0.35994154]
 [0.24933751 1.1245457  0.5868766 ]
 [0.20294677 0.92506635 0.4036633 ]], shape=(3, 3), dtype=float32)


## 

데코레이터를 붙인 결과를 확인해보면, 텐서플로 런타임시의 모든 상호작용들을 다룰 수 있다는 것을 알 수 있습니다

In [5]:
simple_nn_layer

In [6]:
@tf.function
def simple_nn_layer(x, y):
    return tf.nn.relu(tf.matmul(x, y))


x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))


print("On GPU :")
with tf.device('/device:GPU:0'):
    print(simple_nn_layer(x, y))

On GPU :
tf.Tensor(
[[1.277611   0.19934599 1.0558751 ]
 [0.82836896 0.34062582 0.8867041 ]
 [1.7166231  0.60004926 1.6608222 ]], shape=(3, 3), dtype=float32)


In [7]:
simple_nn_layer

In [8]:

def simple_nn_layer(x, y):
    return tf.nn.relu(tf.matmul(x, y))

x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))


print("On GPU :")
with tf.device('/device:GPU:0'):
    print(simple_nn_layer(x, y))

On GPU :
tf.Tensor(
[[0.83025265 1.2190292  1.4702296 ]
 [0.6016683  0.6931868  0.8341508 ]
 [1.0962827  1.5138812  1.5616922 ]], shape=(3, 3), dtype=float32)


In [9]:
simple_nn_layer

<function __main__.simple_nn_layer(x, y)>

## 데코레이터는 선택적

만일 여러분의 코드가 여러 함수들을 포함하고 있다면, 그것들에 모두 데코레이터를 붙일 필요는 없습니다.
데코레이터가 붙은 함수로부터 호출된 모든 함수들은 그래프 모드에서 동작합니다.



In [10]:
def linear_layer(x):
    return 2 * x + 1


In [11]:
@tf.function
def deep_net(x):
    return tf.nn.relu(linear_layer(x))


In [12]:
deep_net(tf.constant((1, 2, 3)))

<tf.Tensor: id=90, shape=(3,), dtype=int32, numpy=array([3, 5, 7])>

## 데코레이터아 일반 함수의 차이

작은 연산들을 많이 포함한 그래프의 경우 함수들은 즉시 실행 코드 (eager code) 보다 더 빠르게 동작합니다. 



In [19]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

In [20]:
@tf.function
def conv_fn(image):
    return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])
# 데이터 준비 (warm up)
conv_layer(image);conv_fn(image)

<tf.Tensor: id=873, shape=(1, 198, 198, 100), dtype=float32, numpy=
array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0.

In [21]:
print("컨볼루션 즉시 실행:", timeit.timeit(lambda: conv_layer(image), number=10))
print("컨볼루션 함수:", timeit.timeit(lambda: conv_fn(image), number=10))
print("컨볼루션의 성능에는 큰 차이가 없다는 것을 확인할 수 있습니다")

컨볼루션 즉시 실행: 0.0035721000000421554
컨볼루션 함수: 0.002928200000042125
컨볼루션의 성능에는 큰 차이가 없다는 것을 확인할 수 있습니다


### 하지만 무거운 연산들을 조금 포함한 그래프의 경우 (컨볼루션 등), 그렇게 빠른 속도 향상은 기대하기 어렵습니다.

In [22]:
lstm_cell = tf.keras.layers.LSTMCell(10)

In [23]:
@tf.function
def lstm_fn(input, state):
    return lstm_cell(input, state)

In [24]:
input = tf.zeros([10, 10])
state = [tf.zeros([10, 10])] * 2
# 데이터 준비 (warm up)
lstm_cell(input, state); lstm_fn(input, state)

(<tf.Tensor: id=1039, shape=(10, 10), dtype=float32, numpy=
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>,
 [<tf.Tensor: id=1040, shape=(10, 10), dtype=float32, numpy=
  array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    

In [25]:
print("lstm 즉시 실행:", timeit.timeit(lambda: lstm_cell(input, state), number=10))
print("lstm 함수:", timeit.timeit(lambda: lstm_fn(input, state), number=10))

lstm 즉시 실행: 0.006141200000001845
lstm 함수: 0.003246100000012575


## 파이썬의 제어 흐름 사용하기

tf.function 내에서 데이터 기반 제어 흐름을 사용할 때, 파이썬의 제어 흐름 문을 사용할 수 있고, 오토그래프 기능은 그것들을 모두 적절한 텐서플로 연산으로 변환할 수 있습니다. 

예를 들어, if 문은 Tensor를 기반으로 작동해야할 때 tf.cond() 로 변환될 수 있습니다.

아래 예시에서, x는 Tensor이지만 if문이 예상한대로 정상 작동합니다:

In [26]:
@tf.function
def square_if_positive(x):
    if x > 0:
        x = x * x
    else:
        x = 0
    return x


print('square_if_positive(2) = {}'.format(square_if_positive(tf.constant(2))))
print('square_if_positive(-2) = {}'.format(square_if_positive(tf.constant(-2))))

square_if_positive(2) = 4
square_if_positive(-2) = 0


### 파이썬 순환문과 제어문 지원

오토그래프는 기본적인 파이썬 문인 while, for, if, break, continue, return과 네스팅(nesting)을 지원합니다.

이는 Tensor 표현을 while과 if 문의 조건 부분에서 사용하거나 for 루프에서 Tensor를 반복할 수 있다는 것을 의미합니다.

In [27]:
@tf.function
def sum_even(items):
    s = 0
    for c in items:
        if c % 2 > 0:
            continue
        s += c
    return s


sum_even(tf.constant([10, 12, 15, 20]))

<tf.Tensor: id=1398, shape=(), dtype=int32, numpy=42>


### 

또한 오토그래프는 고급 사용자를 위해 낮은 수준의 API를 제공합니다. 

예를 들어, 여러분은 생성된 코드를 확인하기 위해 다음과 같이 작성할 수 있습니다.

In [28]:
print(tf.autograph.to_code(sum_even.python_function))

def tf__sum_even(items):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('sum_even', 'sum_even_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as sum_even_scope:
    s = 0

    def get_state_2():
      return ()

    def set_state_2(_):
      pass

    def loop_body(iterates, s):
      c = iterates
      continue_ = False

      def get_state():
        return ()

      def set_state(_):
        pass

      def if_true():
        continue_ = True
        return continue_

      def if_false():
        return continue_
      cond = c % 2 > 0
      continue_ = ag__.if_stmt(cond, if_true, if_false, get_state, set_state, ('continue_',), ())

      def get_state_1():
        return ()

      def set_state_1(_):
        pass

      def if_true_1():
        s_1, = s,
        s_1 += c
        return s_1

      def if_false_1():
        return s
      cond_1 = ag__.not_(continue_)
 

In [30]:
@tf.function
def fizzbuzz(n):
    msg = tf.constant('')
    for i in tf.range(n):
        if tf.equal(i % 3, 0):
            tf.print('Fizz')
        elif tf.equal(i % 5, 0):
            tf.print('Buzz')
        else:
            tf.print(i)

fizzbuzz(tf.constant(15))

Fizz
1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14


## 케라스와 오토그래프

오토그래프는 기본적으로 비동적(non-dynamic) 케라스 모델에서 사용 가능합니다.

In [31]:
class CustomModel(tf.keras.models.Model):

    @tf.function
    def call(self, input_data):
        if tf.reduce_mean(input_data) > 0:
            return input_data
        else:
            return input_data // 2


model = CustomModel()

model(tf.constant([-2, -4]))

<tf.Tensor: id=1507, shape=(2,), dtype=int32, numpy=array([-1, -2])>

## 부수 효과 (Side effects)

즉시 실행 모드 (eager mode)처럼 부수 효과를 사용할 수 있습니다. 예를 들면, tf.function 내에 있는 tf.assign이나 tf.print이 있습니다. 

또한 부수 효과들은 작업들이 순서대로 실행된다는 것을 보장하기 위해 필수적인 제어 의존성 (control dependency)을 추가합니다.

In [32]:
v = tf.Variable(5)

@tf.function
def find_next_odd():
    v.assign(v + 1)
    if tf.equal(v % 2, 0):
        v.assign(v + 1)


find_next_odd()
v

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=7>